<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Assignment:  Machine Learning Prediction


Estimated time needed: **60** minutes


Space X advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against space X for a rocket launch.   In this lab, you will create a machine learning pipeline  to predict if the first stage will land given the data from the preceding labs.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


Most unsuccessful landings are planed. Space X; performs a controlled landing in the oceans.


## Objectives


Perform exploratory  Data Analysis and determine Training Labels

*   create a column for the class
*   Standardize the data
*   Split into training data and test data

\-Find best Hyperparameter for SVM, Classification Trees and Logistic Regression

*   Find the method performs best using test data


## Import Libraries and Define Auxiliary Functions


In [1]:
import piplite
await piplite.install(['numpy'])
await piplite.install(['pandas'])
await piplite.install(['seaborn'])

We will import the following libraries for the lab


In [2]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
# Preprocessing allows us to standarsize our data
from sklearn import preprocessing
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split
# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier

<ipython-input-2-b7d446354769>:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


This function is to plot the confusion matrix.


In [7]:
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed']) 
    plt.show() 

## Load the dataframe


Load the data


In [4]:
from js import fetch
import io

URL1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
resp1 = await fetch(URL1)
text1 = io.BytesIO((await resp1.arrayBuffer()).to_py())
data = pd.read_csv(text1)

In [5]:
data.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


In [6]:
URL2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_3.csv'
resp2 = await fetch(URL2)
text2 = io.BytesIO((await resp2.arrayBuffer()).to_py())
X = pd.read_csv(text2)

In [ ]:
X.head(100)

## TASK  1


Create a NumPy array from the column <code>Class</code> in <code>data</code>, by applying the method <code>to_numpy()</code>  then
assign it  to the variable <code>Y</code>,make sure the output is a  Pandas series (only one bracket df\['name of  column']).


In [9]:
import pandas as pd

# URL for the dataset
URL2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_3.csv'

# Load the dataset
X = pd.read_csv(URL2)

# Check the first few rows to confirm the dataset is loaded
print(X.head())

# TASK 1: Create a NumPy array from the 'Class' column
Y = X['Class'].to_numpy()

# Print the result to confirm
print(Y)


<class 'urllib.error.URLError'>: <urlopen error [Errno 23] Host is unreachable>

## TASK  2


Standardize the data in <code>X</code> then reassign it to the variable  <code>X</code> using the transform provided below.


In [10]:
# students get this 
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the data
# Assuming all columns need to be standardized except non-numeric ones.
# We need to exclude non-numeric columns like 'Date', 'BoosterVersion', 'Orbit', etc.

# Select only numeric columns for standardization (excluding categorical/textual columns)
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Apply the scaler to these numeric columns
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

# Check the standardized data
print(X.head())

transform = preprocessing.StandardScaler()

   FlightNumber   PayloadMass   Flights     Block  ReusedCount  Orbit_ES-L1  \
0     -1.712912 -1.948145e-16 -0.653913 -1.575895     -0.97344       -0.106   
1     -1.674419 -1.195232e+00 -0.653913 -1.575895     -0.97344       -0.106   
2     -1.635927 -1.162673e+00 -0.653913 -1.575895     -0.97344       -0.106   
3     -1.597434 -1.200587e+00 -0.653913 -1.575895     -0.97344       -0.106   
4     -1.558942 -6.286706e-01 -0.653913 -1.575895     -0.97344       -0.106   

   Orbit_GEO  Orbit_GTO  Orbit_HEO  Orbit_ISS  ...  Serial_B1058  \
0     -0.106  -0.654654     -0.106  -0.551677  ...     -0.185695   
1     -0.106  -0.654654     -0.106  -0.551677  ...     -0.185695   
2     -0.106  -0.654654     -0.106   1.812654  ...     -0.185695   
3     -0.106  -0.654654     -0.106  -0.551677  ...     -0.185695   
4     -0.106   1.527525     -0.106  -0.551677  ...     -0.185695   

   Serial_B1059  Serial_B1060  Serial_B1062  GridFins_False  GridFins_True  \
0     -0.215666     -0.185695        -

We split the data into training and testing data using the  function  <code>train_test_split</code>.   The training data is divided into validation data, a second set used for training  data; then the models are trained and hyperparameters are selected using the function <code>GridSearchCV</code>.


## TASK  3


Use the function train_test_split to split the data X and Y into training and test data. Set the parameter test_size to  0.2 and random_state to 2. The training data and test data should be assigned to the following labels.


<code>X_train, X_test, Y_train, Y_test</code>


In [16]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
# X is the feature set, Y is the target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Check the shape of the resulting splits to confirm
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

<class 'NameError'>: name 'Y' is not defined

we can see we only have 18 test samples.


In [12]:
Y_test.shape

<class 'NameError'>: name 'Y_test' is not defined

## TASK  4


Create a logistic regression object  then create a  GridSearchCV object  <code>logreg_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [13]:
parameters ={"C":[0.01,0.1,1],'penalty':['l2'], 'solver':['lbfgs']}# l1 lasso l2 ridge
lr=LogisticRegression()
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Define the logistic regression object
lr = LogisticRegression()

# Define the parameters grid
parameters = {
    'C': [0.01, 0.1, 1],
    'penalty': ['l2'],  # 'l2' is the ridge penalty
    'solver': ['lbfgs']  # 'lbfgs' solver supports 'l2' penalty
}

# Create the GridSearchCV object with 10-fold cross-validation
logreg_cv = GridSearchCV(lr, parameters, cv=10)

# Fit the GridSearchCV object to the training data
logreg_cv.fit(X_train, Y_train)

# Output the best parameters and the best score
print("Best Parameters: ", logreg_cv.best_params_)
print("Best Score: ", logreg_cv.best_score_)


<class 'NameError'>: name 'X_train' is not defined

We output the <code>GridSearchCV</code> object for logistic regression. We display the best parameters using the data attribute <code>best_params\_</code> and the accuracy on the validation data using the data attribute <code>best_score\_</code>.


In [14]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

<class 'AttributeError'>: 'GridSearchCV' object has no attribute 'best_params_'

## TASK  5


Calculate the accuracy on the test data using the method <code>score</code>:


In [15]:
# Calculate the accuracy on the test data
accuracy = logreg_cv.score(X_test, Y_test)

# Print the accuracy
print("Accuracy on test data: {:.2f}%".format(accuracy * 100))


<class 'NameError'>: name 'X_test' is not defined

Lets look at the confusion matrix:


In [17]:
yhat=logreg_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

<class 'NameError'>: name 'X_test' is not defined

Examining the confusion matrix, we see that logistic regression can distinguish between the different classes.  We see that the problem is false positives.

Overview:

True Postive - 12 (True label is landed, Predicted label is also landed)

False Postive - 3 (True label is not landed, Predicted label is landed)


## TASK  6


Create a support vector machine object then  create a  <code>GridSearchCV</code> object  <code>svm_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}
svm = SVC()

In [18]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the support vector machine object
svm = SVC()

# Define the parameters for the grid search
parameters = {
    'kernel': ('linear', 'rbf', 'poly', 'sigmoid'),
    'C': np.logspace(-3, 3, 5),  # C values from 0.001 to 1000
    'gamma': np.logspace(-3, 3, 5)  # gamma values from 0.001 to 1000
}

# Create the GridSearchCV object with 10-fold cross-validation
svm_cv = GridSearchCV(svm, parameters, cv=10)

# Fit the GridSearchCV object to the training data
svm_cv.fit(X_train, Y_train)

# Output the best parameters and the best score
print("Best Parameters: ", svm_cv.best_params_)
print("Best Score: ", svm_cv.best_score_)


<class 'NameError'>: name 'X_train' is not defined

In [19]:
print("tuned hpyerparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy :",svm_cv.best_score_)

<class 'AttributeError'>: 'GridSearchCV' object has no attribute 'best_params_'

## TASK  7


Calculate the accuracy on the test data using the method <code>score</code>:


In [20]:
# Calculate the accuracy on the test data using the best model from GridSearchCV
accuracy_svm = svm_cv.score(X_test, Y_test)

# Print the accuracy
print("Accuracy on test data (SVM): {:.2f}%".format(accuracy_svm * 100))


<class 'NameError'>: name 'X_test' is not defined

We can plot the confusion matrix


In [21]:
yhat=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

<class 'NameError'>: name 'X_test' is not defined

## TASK  8


Create a decision tree classifier object then  create a  <code>GridSearchCV</code> object  <code>tree_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeClassifier()

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Create the decision tree classifier object
tree = DecisionTreeClassifier()

# Define the parameter grid for tuning
parameters = {
    'criterion': ['gini', 'entropy'],  # Criterion for splitting
    'splitter': ['best', 'random'],  # Strategy for splitting at nodes
    'max_depth': [2*n for n in range(1, 10)],  # Maximum depth of the tree
    'max_features': ['auto', 'sqrt'],  # Number of features to consider for best split
    'min_samples_leaf': [1, 2, 4],  # Minimum samples per leaf node
    'min_samples_split': [2, 5, 10]  # Minimum number of samples to split a node
}

# Create the GridSearchCV object with 10-fold cross-validation
tree_cv = GridSearchCV(tree, parameters, cv=10)

# Fit the GridSearchCV object to the training data
tree_cv.fit(X_train, Y_train)

# Output the best parameters and the best score
print("Best Parameters: ", tree_cv.best_params_)
print("Best Score: ", tree_cv.best_score_)


<class 'NameError'>: name 'X_train' is not defined

In [23]:
print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy :",tree_cv.best_score_)

<class 'AttributeError'>: 'GridSearchCV' object has no attribute 'best_params_'

## TASK  9


Calculate the accuracy of tree_cv on the test data using the method <code>score</code>:


In [24]:
# Calculate the accuracy of the best decision tree model on the test data
accuracy_tree = tree_cv.score(X_test, Y_test)

# Print the accuracy
print("Accuracy on test data (Decision Tree): {:.2f}%".format(accuracy_tree * 100))


<class 'NameError'>: name 'X_test' is not defined

We can plot the confusion matrix


In [25]:
yhat = tree_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

<class 'NameError'>: name 'X_test' is not defined

## TASK  10


Create a k nearest neighbors object then  create a  <code>GridSearchCV</code> object  <code>knn_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

KNN = KNeighborsClassifier()

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Create the K-Nearest Neighbors classifier object
KNN = KNeighborsClassifier()

# Define the parameter grid for tuning
parameters = {
    'n_neighbors': list(range(1, 11)),  # Number of neighbors to use
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
    'p': [1, 2]  # Power parameter for the Minkowski distance (1 for Manhattan, 2 for Euclidean)
}

# Create the GridSearchCV object with 10-fold cross-validation
knn_cv = GridSearchCV(KNN, parameters, cv=10)

# Fit the GridSearchCV object to the training data
knn_cv.fit(X_train, Y_train)

# Output the best parameters and the best score
print("Best Parameters: ", knn_cv.best_params_)
print("Best Score: ", knn_cv.best_score_)


<class 'NameError'>: name 'X_train' is not defined

In [27]:
print("tuned hpyerparameters :(best parameters) ",knn_cv.best_params_)
print("accuracy :",knn_cv.best_score_)

<class 'AttributeError'>: 'GridSearchCV' object has no attribute 'best_params_'

## TASK  11


Calculate the accuracy of knn_cv on the test data using the method <code>score</code>:


In [28]:
# Calculate the accuracy of the best KNN model on the test data
accuracy_knn = knn_cv.score(X_test, Y_test)

# Print the accuracy
print("Accuracy on test data (KNN): {:.2f}%".format(accuracy_knn * 100))


<class 'NameError'>: name 'X_test' is not defined

We can plot the confusion matrix


In [29]:
yhat = knn_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

<class 'NameError'>: name 'X_test' is not defined

## TASK  12


Find the method performs best:


In [30]:
# Assuming you have accuracy values for each model
accuracy_logreg = logreg_cv.score(X_test, Y_test)
accuracy_svm = svm_cv.score(X_test, Y_test)
accuracy_tree = tree_cv.score(X_test, Y_test)
accuracy_knn = knn_cv.score(X_test, Y_test)

# Print the accuracy for each model
print("Accuracy on test data (Logistic Regression): {:.2f}%".format(accuracy_logreg * 100))
print("Accuracy on test data (SVM): {:.2f}%".format(accuracy_svm * 100))
print("Accuracy on test data (Decision Tree): {:.2f}%".format(accuracy_tree * 100))
print("Accuracy on test data (KNN): {:.2f}%".format(accuracy_knn * 100))

# Find and print the best performing method
accuracies = {
    'Logistic Regression': accuracy_logreg,
    'SVM': accuracy_svm,
    'Decision Tree': accuracy_tree,
    'KNN': accuracy_knn
}

best_method = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_method]

print("\nBest performing method: {} with accuracy: {:.2f}%".format(best_method, best_accuracy * 100))


<class 'NameError'>: name 'X_test' is not defined

## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
